In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 15)

matplotlib = 3.3.2

numpy = 1.19.2

pandas = 1.1.3

psycopg2 = 2.8.5 

python = 3.8.5

In [2]:
df = pd.read_csv('C:/Users/Kate/PycharmProjects/test/test/P9-ConsumerComplaints.csv',
                 header = 0,
                 na_values=['N/A'],
                 index_col = 'Complaint ID',
                 true_values=['Yes'],
                 false_values=['No'],
#                  keep_default_na=False,
                 low_memory=False
                 )
df.head()

,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed
Complaint ID,,,,,,,,,,,,,,,,,
468882,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,2013-07-30,Closed with explanation,True,False
468889,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,True,False
468879,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,2013-07-31,Closed,True,False
468949,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,GA,30084,NaN,NaN,Web,2013-07-30,Closed with explanation,True,False
475823,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Franklin Credit Management,CT,6106,NaN,NaN,Web,2013-07-30,Closed with explanation,True,False


In [3]:
## import via url is also available

# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
# df = pd.read_csv('https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P9-ConsumerComplaints.csv', low_memory=False)


In [4]:
df['Date Received'] = pd.to_datetime(df['Date Received'])

In [5]:
df.dtypes

Date Received                   datetime64[ns]
Product Name                            object
Sub Product                             object
Issue                                   object
Sub Issue                               object
                                     ...      
Submitted via                           object
Date Sent to Company                    object
Company Response to Consumer            object
Timely Response                           bool
Consumer Disputed                       object
Length: 17, dtype: object

### 1. 

Write a program which takes date range as an input and displays for that range list of all product names, amount of issues (complaints) for each product name, how much issues were done with timely response and how much were disputed by customer. Display results sorted by amount of issues in descending order.



In [6]:
date_start = pd.to_datetime('2013-07-29')
date_end = pd.to_datetime('2013-07-30')

In [7]:
# parse NA values in timely response column

df.loc[df['Timely Response'] != True, 'TR False NA'] = False  
df.loc[df['Timely Response'] == True, 'TR False NA'] = True

In [8]:
# parse NA value in consumer disputed column

df.loc[df['Consumer Disputed'] != True, 'CD False NA'] = False  
df.loc[df['Consumer Disputed'] == True, 'CD False NA'] = True

In [9]:
# counting number of timely responses for each issue in the given date range

tr_ic_df = df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['TR False NA'] == True)][['Product Name', 'Issue', 'TR False NA']]\
.groupby(['Product Name', 'Issue'])\
.count()


In [10]:
tr_ic_df.head()

TR False NA
Product Name            Issue                                                
Bank account or service Account opening, closing, or management            12
                        Deposits and withdrawals                           13
                        Making/receiving payments, sending money            2
                        Problems caused by my funds being low               4
                        Using a debit or ATM card                           3

In [11]:
# counting number of consumer disputed for each issue in the given date range

cd_ic_df = df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['CD False NA'] == True)][['Product Name', 'Issue', 'CD False NA']]\
.groupby(['Product Name', 'Issue'])\
.count()

In [12]:
cd_ic_df.head()

CD False NA
Product Name            Issue                                               
Bank account or service Account opening, closing, or management            3
Consumer Loan           Managing the loan or lease                         3
Credit card             APR or interest rate                               1
                        Billing disputes                                   2
                        Billing statement                                  1

In [13]:
# full join on index
res = tr_ic_df.merge(cd_ic_df, left_index=True, right_index=True, how='outer').fillna(0)
res['CD False NA'] = res['CD False NA'].astype('int64', errors='raise')

In [14]:
res.head()

TR False NA  \
Product Name            Issue                                                   
Bank account or service Account opening, closing, or management            12   
                        Deposits and withdrawals                           13   
                        Making/receiving payments, sending money            2   
                        Problems caused by my funds being low               4   
                        Using a debit or ATM card                           3   

                                                                  CD False NA  
Product Name            Issue                                                  
Bank account or service Account opening, closing, or management             3  
                        Deposits and withdrawals                            0  
                        Making/receiving payments, sending money            0  
                        Problems caused by my funds being low               0  
                        Using a debit or ATM card                           0

In [15]:
# rename columns to compare two tables
pd_res_df = res
pd_res_df.index.names = ['product_name', 'issue']
pd_res_df.columns = ['trc', 'cdc']

In [16]:
pd_res_df.head()

trc  cdc
product_name            issue                                             
Bank account or service Account opening, closing, or management    12    3
                        Deposits and withdrawals                   13    0
                        Making/receiving payments, sending money    2    0
                        Problems caused by my funds being low       4    0
                        Using a debit or ATM card                   3    0

In [42]:
import psycopg2
from dbkeys import USER_DB, PASS_DB, HOST_DB, PORT_DB, DB_NAME

connection = psycopg2.connect(user=USER_DB,
                              password=PASS_DB,
                              host=HOST_DB,
                              port=PORT_DB,
                              database=DB_NAME)



In [18]:
ds = "'" + date_start.strftime('%Y-%m-%d') + "'"
de = "'" + date_end.strftime('%Y-%m-%d') + "'"

query = """
        select  product_name, issue,
        count(timely_response) filter (where timely_response_bool is true) trc,
        count(consumer_disputed) filter (where consumer_disputed_bool is true) cdc
        from user_complaints
        where date_received >= {ds} and date_received <= {de}
        group by product_name, issue
        order by product_name, issue, count(issue) desc
    ;""".format(ds = ds, de = de)
sql_res_df = pd.read_sql(query, connection, parse_dates={'date_received': {'format': '%Y-%m-%d'}}, index_col = ['product_name', 'issue'])


In [19]:
sql_res_df.head()

trc  cdc
product_name            issue                                             
Bank account or service Account opening, closing, or management    12    3
                        Deposits and withdrawals                   13    0
                        Making/receiving payments, sending money    2    0
                        Problems caused by my funds being low       4    0
                        Using a debit or ATM card                   3    0

In [23]:
# performance test

import time
m_start = time.time()

tr_ic_df = df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['TR False NA'] == True)][['Product Name', 'Issue', 'TR False NA']]\
.groupby(['Product Name', 'Issue'])\
.count()

cd_ic_df = df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['CD False NA'] == True)][['Product Name', 'Issue', 'CD False NA']]\
.groupby(['Product Name', 'Issue'])\
.count()

res = tr_ic_df.merge(cd_ic_df, left_index=True, right_index=True, how='outer').fillna(0)
res['CD False NA'] = res['CD False NA'].astype(int, errors='raise')

m_end = time.time()

print('pandas query execution time %.4f s'%(m_end - m_start))

m_start = time.time()

query = """
        select  product_name, issue,
        count(timely_response) filter (where timely_response_bool is true) trc,
        count(consumer_disputed) filter (where consumer_disputed_bool is true) cdc
        from user_complaints
        where date_received >= {ds} and date_received <= {de}
        group by product_name, issue
        order by product_name, issue, count(issue) desc
    ;""".format(ds = ds, de = de)
sql_res_df = pd.read_sql(query, connection, parse_dates={'date_received': {'format': '%Y-%m-%d'}}, index_col = ['product_name', 'issue'])

m_end = time.time()

print('postgresql query execution time %.4f s'%(m_end - m_start))

pandas query execution time 0.0400 s
postgresql query execution time 0.0350 s


### 2.

Write a program which takes company name as an input, finds state name with biggest amount of issues for that company (filter out empty state values) and displays list of all issues (with all related attributes) for that company and state name.

In [28]:
# the second query
# which state has the biggest amount of issues for Encore Capital Group

bad_company = 'Encore Capital Group'

bad_state = df[df['Company'] == bad_company][['State Name', 'Issue']]\
.groupby(['State Name'])\
.count()\
.sort_values(by = 'Issue', ascending = False)\
.iloc[0].name

In [29]:
bad_state

'CA'

In [34]:

df[(df['Company'] == bad_company) &\
   (df['State Name'] == bad_state)][['Company', 'State Name', 'Product Name', 'Sub Product', 'Issue']]\
.groupby(['Company', 'State Name', 'Product Name', 'Sub Product', 'Issue'])\
.count()


Empty DataFrame
Columns: []
Index: [(Encore Capital Group, CA, Consumer Loan, Installment loan, Managing the loan or lease), (Encore Capital Group, CA, Consumer Loan, Installment loan, Problems when you are unable to pay), (Encore Capital Group, CA, Consumer Loan, Personal line of credit, Account terms and changes), (Encore Capital Group, CA, Debt collection, Credit card, Communication tactics), (Encore Capital Group, CA, Debt collection, Credit card, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Credit card, Disclosure verification of debt), (Encore Capital Group, CA, Debt collection, Credit card, False statements or representation), (Encore Capital Group, CA, Debt collection, Credit card, Improper contact or sharing of info), (Encore Capital Group, CA, Debt collection, Credit card, Taking/threatening an illegal action), (Encore Capital Group, CA, Debt collection, I do not know, Communication tactics), (Encore Capital Group, CA, Debt collection, I do not know, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, I do not know, Disclosure verification of debt), (Encore Capital Group, CA, Debt collection, I do not know, False statements or representation), (Encore Capital Group, CA, Debt collection, I do not know, Improper contact or sharing of info), (Encore Capital Group, CA, Debt collection, Medical, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Mortgage, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Non-federal student loan, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Communication tactics), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Disclosure verification of debt), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), False statements or representation), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Improper contact or sharing of info), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Taking/threatening an illegal action)]

[23 rows x 0 columns]

In [37]:

query = f"""
with t1 as (
select
    company, state_name, count(issue) ci 
from user_complaints
where company = '{bad_company}'
group by company, state_name
order by ci desc
limit 1)
select 
t1.company, t1.state_name, uc.product_name, uc.sub_product, uc.issue
from user_complaints uc, t1
where uc.company = t1.company and uc.state_name = t1.state_name
group by t1.company, t1.state_name, uc.product_name, uc.sub_product, uc.issue       
;"""
        
sql2_res_df = pd.read_sql(query, connection)
sql2_res_df.head()

,company,state_name,product_name,sub_product,issue
0,Encore Capital Group,CA,Consumer Loan,Installment loan,Managing the loan or lease
1,Encore Capital Group,CA,Consumer Loan,Installment loan,Problems when you are unable to pay
2,Encore Capital Group,CA,Consumer Loan,Personal line of credit,Account terms and changes
3,Encore Capital Group,CA,Credit card,,Customer service / Customer relations
4,Encore Capital Group,CA,Credit card,,Other


In [44]:
# performance test 2

m_start = time.time()

bad_state = df[df['Company'] == bad_company][['State Name', 'Issue']]\
.groupby(['State Name'])\
.count()\
.sort_values(by = 'Issue', ascending = False)\
.iloc[0].name

pd2_res_df = df[(df['Company'] == bad_company) &\
   (df['State Name'] == bad_state)][['Company', 'State Name', 'Product Name', 'Sub Product', 'Issue']]\
.groupby(['Company', 'State Name', 'Product Name', 'Sub Product', 'Issue'])\
.count()

m_end = time.time()

print('pandas query execution time %.4f s'%(m_end - m_start))

query = f"""
with t1 as (
select
    company, state_name, count(issue) ci 
from user_complaints
where company = '{bad_company}'
group by company, state_name
order by ci desc
limit 1)
select 
t1.company, t1.state_name, uc.product_name, uc.sub_product, uc.issue
from user_complaints uc, t1
where uc.company = t1.company and uc.state_name = t1.state_name
group by t1.company, t1.state_name, uc.product_name, uc.sub_product, uc.issue       
;"""

m_start = time.time()

sql2_res_df = pd.read_sql(query, connection)

m_end = time.time()

print('postgresql query execution time %.4f s'%(m_end - m_start))

pandas query execution time 0.0290 s
postgresql query execution time 0.0730 s


In [40]:
if (connection):
    connection.close()